# Lab 4: Recurrent models

This lab gives you practice with embeddings (word vectors, in this case) and recurrent neural models in NLP. The first part focuses on embeddings as input to a recurrent model, and the second part focuses on embeddings derived from recurrent models, applying them to the task of word sense disambuiguation following the approach from the original paper by Peters et al.

Everybody's machine is different and the neural computations required for this lab are more demanding than in the other assignments in this course. For this reason, it is advisable to use Google CoLab which guarantees a minimum level of performance. It is also recommended to use GPU acceleration; on CoLab, it can be turned on via <code>Runtime>Change Runtime type>GPU</code>.

## Part 1 (45 points)

In the first part of lab 4, we will play with training a recurrent model for part of speech tagging. As an easy exercise, you will observe what happens when you plug in pretrained word embeddings into a neural NLP model and experiment with different sizes of training data.

If you use Google Colab, it may be easiest to place this notebook and <code>lstm_tutorial.py</code> in <code>/Colab Notebooks</code> directory of your Google Drive. Run the code in the cell just below to enable CoLab to access the files on Drive. This will produce a link to an obscure redirect page which misleadingly suggests that you are getting Google Drive Desktop installation; what happens instead you are allowing the notebook running on Colab access to Drive. Copy the code that appears after you allow Drive access and paste it below.

In [3]:
#RUN THIS CELL IF USING COLAB TO USE GOOGLE DRIVE FOR STORING lstm_tutorial.py AND/OR DATA FILES
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My Drive/NLP/Lab4/

Mounted at /content/drive
/content/drive/My Drive/NLP/Lab4


The neural network solutions in this lab rely on AllenNLP library version 0.9.0 (the other code of this lab assignment may work incorrectly with more recent versions); <code>overrides</code> is required for compatibility. Linguistic resources are from NLTK version 3.6.2 and might work incorrectly in other versions. Install these before proceeding; installation process may vary depending on your system. On CoLab, this can be done via the following command:

In [4]:
#IF USING COLAB, INSTALL allennlp AND nltk AS FOLLOWS
!pip install -U allennlp==0.9.0 overrides==3.1.0 nltk==3.6.2

     |████████████████████████████████| 7.6MB 4.3MB/s 
     |████████████████████████████████| 1.5MB 33.9MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 133kB 41.1MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 266kB 37.4MB/s 
     |████████████████████████████████| 245kB 34.1MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 133kB 40.0MB/s 
     |████████████████████████████████| 122kB 39.7MB/s 
     |████████████████████████████████| 163kB 36.6MB/s 
     |████████████████████████████████| 5.6MB 35.2MB/s 
     |████████████████████████████████| 30.8MB 96kB/s 
     |████████████████████████████████| 143kB 37.2MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 7.6MB 32.3MB/s 
     |████████████████████████████████| 1.2MB 28.2MB/s 
     |████████████████████████████████| 256kB 37.7MB/s 
  

**Before you start**,  import required modules:

In [5]:
import random
import nltk
import allennlp

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


If you run this for the first time, you may need to download various data using NLTK:

In [ ]:
nltk.download('brown')
nltk.download('semcor')
nltk.download('wordnet')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Exercise 1: prepare the data (5 points)

Linguistic data come in a variety of formats. You already had a chance to play with POS-annotated corpus data in Lab 1.

In the first exercise, you will access POS-annotated data in one format (NLTK) and save it on the disk in a text format. Start with the tagged sentences from the Brown corpus, which can be retrieved as below:

In [ ]:
nltk.corpus.brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

Now randomize the order of all sentences in the corpus using <code>random.shuffle()</code> function and select the first 50K sentences for training and the next 5K for validation.

In [ ]:
data = list(nltk.corpus.brown.tagged_sents())

random.shuffle(data)

training_brown = data[:50000]
validation_brown = data[50000:]
testing_brown = []

In [ ]:
len(training_brown)

50000

Define a function for saving your datasets to a text file in the following format:
* one sentence per line
* tokens separated by spaces
* POS tag separated from the token by "###", for example <code>said###VBD</code>.

In [ ]:
def write_posdata(sentences, fileName):
  outfile = open(fileName, 'w')
  output = []
  for sentence in sentences:
    for word, pos in sentence:
      output.append(word + "###" + pos + " ")
    output.append("\r\n")
    
  outfile.writelines(output)
  outfile.close()

Now save your data partitions in different sizes. We will start with small data samples since training on a large dataset may be very slow depending on your machine. We won't use the full 50K sentence training set in this lab since this might take too long.

In [ ]:
write_posdata(training_brown,"train_brown.txt")
write_posdata(validation_brown,"validation_brown.txt")
write_posdata(training_brown[:50],"train_brown_50.txt")
write_posdata(validation_brown[:50],"validation_brown_50.txt")
write_posdata(training_brown[:500],"train_brown_500.txt")
write_posdata(validation_brown[:500],"validation_brown_500.txt")
write_posdata(training_brown[:5000],"train_brown_5000.txt")
write_posdata(validation_brown[:5000],"validation_brown_5000.txt")

Congratulations, you have now saved the POS tagged data for model training purposes!

## Exercise 2: train neural POS tagger models (15 points)

We will now play with a neural model. You have installed <code>allennlp</code> which contains all necessary components for this and the training code for an LSTM model, which follows an old AllenNLP tutorial, is contained in <code>lstm_tutorial.py</code>. PLace the latter in the same directory as this notebook. Let us start by loading the model code and data, starting with a tiny sample for demonstration purposes. 

In [12]:
from lstm_tutorial import *

train_dataset_tiny = reader.read("train_brown_50.txt")
validation_dataset_tiny = reader.read("validation_brown_50.txt")

50it [00:00, 67.48it/s]
50it [00:00, 102.26it/s]


Fist of all we need to initialize the vocabulary and define an embedding (vector) for each token. We set the embedding size at 300, common in realistic applications. By default, the embeddings are initialized randomly and updated during trining (this can be changed but we start with a standard configuration). We also need to specify the <code>HIDDEN_DIM</code> parameter: the dimensionality of the hidden vector representations in the LSTM cell.

In [ ]:
vocab_tiny = Vocabulary.from_instances(train_dataset_tiny + validation_dataset_tiny)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

token_embedding_tiny = Embedding(num_embeddings=vocab_tiny.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

100%|██████████| 100/100 [00:00<00:00, 43027.33it/s]


Download the smallest pretrained word vector model from https://nlp.stanford.edu/projects/glove/, unzip it, and extract the relevant file <code>'glove.6B.300d.txt'</code> in your working directory. The size of the file is 1GB; if using Google Drive with Colab, make sure you have sufficient space. Downloading and uploading the file might take a few minutes. You can <b>either</b> upload the relevant file from your personal machine `or` use the code below directly from CoLab:

In [ ]:
#THIS CELL IS OPTIONAL, TO BE USED ON COLAB. YOU CAN USE wget AS BELOW OR ALTERNATIVELY UPLOAD GloVe EMBEDDINGS TO GOOGLE DRIVE FROM YOUR MACHINE
# download the file
!wget http://nlp.stanford.edu/data/glove.6B.zip
# unzip the file
!unzip -d . 'glove.6B.zip'
# remove useless contents
!rm 'glove.6B.200d.txt' 'glove.6B.100d.txt' 'glove.6B.50d.txt'

--2021-06-21 11:10:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-21 11:10:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-21 11:10:53--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Initialize token embeddings with values from pretrained GloVe model:


In [ ]:
glove_token_embedding_tiny = Embedding.from_params(vocab=vocab_tiny,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))

400000it [00:03, 113116.37it/s]


Now from embedding a single word with <code>token_embedding_tiny</code> we can proceed to mapping a word sequence into a sequence of vectors:

In [ ]:
word_embeddings_tiny = BasicTextFieldEmbedder({"tokens": token_embedding_tiny})

The following initializes parameters of an LSTM model using <code>word_embeddings_tiny</code> input encoding

In [ ]:
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model_tiny = LstmTagger(word_embeddings_tiny, lstm, vocab_tiny)

Now define an LSTM model called <code>glove_model_tiny</code> that uses <code>glove_token_embedding_tiny</code>:

In [ ]:
glove_model_tiny = LstmTagger(BasicTextFieldEmbedder({"tokens": glove_token_embedding_tiny}), lstm, vocab_tiny)

Train the basic model for the tiny dataset. **Do not clear the output of this cell in the submitted version.**



In [ ]:
basic_trainer_tiny=initialize_trainer(model_tiny,vocab_tiny,train_dataset_tiny,validation_dataset_tiny,batch_size=50)
basic_trainer_tiny.train()

accuracy: 0.0154, loss: 4.5609 ||: 100%|██████████| 1/1 [00:00<00:00,  2.78it/s]
accuracy: 0.0210, loss: 4.5441 ||: 100%|██████████| 1/1 [00:00<00:00, 44.98it/s]
accuracy: 0.0154, loss: 4.5556 ||: 100%|██████████| 1/1 [00:00<00:00, 29.73it/s]
accuracy: 0.0220, loss: 4.5391 ||: 100%|██████████| 1/1 [00:00<00:00, 52.73it/s]
accuracy: 0.0144, loss: 4.5504 ||: 100%|██████████| 1/1 [00:00<00:00, 32.24it/s]
accuracy: 0.0210, loss: 4.5341 ||: 100%|██████████| 1/1 [00:00<00:00, 46.75it/s]
accuracy: 0.0165, loss: 4.5451 ||: 100%|██████████| 1/1 [00:00<00:00, 31.79it/s]
accuracy: 0.0199, loss: 4.5291 ||: 100%|██████████| 1/1 [00:00<00:00, 55.67it/s]
accuracy: 0.0175, loss: 4.5399 ||: 100%|██████████| 1/1 [00:00<00:00, 31.04it/s]
accuracy: 0.0189, loss: 4.5241 ||: 100%|██████████| 1/1 [00:00<00:00, 60.46it/s]
accuracy: 0.0185, loss: 4.5347 ||: 100%|██████████| 1/1 [00:00<00:00, 33.17it/s]
accuracy: 0.0252, loss: 4.5192 ||: 100%|██████████| 1/1 [00:00<00:00, 55.53it/s]
accuracy: 0.0206, loss: 4.52

{'best_epoch': 999,
 'best_validation_accuracy': 0.38155136268343814,
 'best_validation_loss': 3.0422229766845703,
 'epoch': 999,
 'peak_cpu_memory_MB': 2267.744,
 'peak_gpu_0_memory_MB': 507,
 'training_accuracy': 0.3903192584963955,
 'training_cpu_memory_MB': 2267.744,
 'training_duration': '0:02:26.720229',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 507,
 'training_loss': 2.779458522796631,
 'training_start_epoch': 0,
 'validation_accuracy': 0.38155136268343814,
 'validation_loss': 3.0422229766845703}

You have trained an LSTM POS tagger for the basic model. Now train the <code>glove_model_tiny</code>. **Do not clear the output of this cell in the submitted version.**

In [ ]:
glove_trainer_tiny=initialize_trainer(glove_model_tiny,vocab_tiny,train_dataset_tiny,validation_dataset_tiny,batch_size=50)
glove_trainer_tiny.train()

accuracy: 0.0021, loss: 4.5843 ||: 100%|██████████| 1/1 [00:00<00:00, 32.04it/s]
accuracy: 0.0010, loss: 4.5724 ||: 100%|██████████| 1/1 [00:00<00:00, 67.75it/s]
accuracy: 0.0021, loss: 4.5571 ||: 100%|██████████| 1/1 [00:00<00:00, 30.56it/s]
accuracy: 0.0000, loss: 4.5479 ||: 100%|██████████| 1/1 [00:00<00:00, 47.37it/s]
accuracy: 0.0021, loss: 4.5300 ||: 100%|██████████| 1/1 [00:00<00:00, 33.15it/s]
accuracy: 0.0000, loss: 4.5235 ||: 100%|██████████| 1/1 [00:00<00:00, 58.20it/s]
accuracy: 0.0021, loss: 4.5030 ||: 100%|██████████| 1/1 [00:00<00:00, 29.21it/s]
accuracy: 0.0000, loss: 4.4991 ||: 100%|██████████| 1/1 [00:00<00:00, 48.14it/s]
accuracy: 0.0031, loss: 4.4760 ||: 100%|██████████| 1/1 [00:00<00:00, 29.06it/s]
accuracy: 0.0031, loss: 4.4748 ||: 100%|██████████| 1/1 [00:00<00:00, 59.25it/s]
accuracy: 0.0062, loss: 4.4491 ||: 100%|██████████| 1/1 [00:00<00:00, 30.68it/s]
accuracy: 0.0157, loss: 4.4505 ||: 100%|██████████| 1/1 [00:00<00:00, 54.62it/s]
accuracy: 0.0134, loss: 4.42

{'best_epoch': 999,
 'best_validation_accuracy': 0.4790356394129979,
 'best_validation_loss': 2.3507449626922607,
 'epoch': 999,
 'peak_cpu_memory_MB': 2267.964,
 'peak_gpu_0_memory_MB': 507,
 'training_accuracy': 0.6570545829042225,
 'training_cpu_memory_MB': 2267.964,
 'training_duration': '0:02:26.034040',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 507,
 'training_loss': 1.453727126121521,
 'training_start_epoch': 0,
 'validation_accuracy': 0.4790356394129979,
 'validation_loss': 2.3507449626922607}

## Exercise 3: Explore training parameters (25 points)

Create separate models on the basis of bigger datasets: the 500 sentence training and 500 sentence validation and 5000 sentence training and 5000 sentence validation. Using the full training set (50K sentences) is optional (your machine might be too slow). Initialize and train the basic model on 500 sentence training and 500 sentence validation data. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the basic model on 500 sentences
train_data_tiny_500 = reader.read("train_brown_500.txt")
val_data_tiny_500 = reader.read("validation_brown_500.txt")

vocab_tiny_500 = Vocabulary.from_instances(train_data_tiny_500 + val_data_tiny_500)

token_embedding_tiny_500 = Embedding(vocab_tiny_500.get_vocab_size(), EMBEDDING_DIM)

model_tiny_500 = LstmTagger(BasicTextFieldEmbedder({"tokens": token_embedding_tiny_500}), lstm, vocab_tiny_500)

basic_trainer_tiny_500 = initialize_trainer(model_tiny_500,vocab_tiny_500,train_data_tiny_500,val_data_tiny_500,batch_size=50)
basic_trainer_tiny_500.train()

500it [00:00, 14334.89it/s]
500it [00:00, 15245.14it/s]
100%|██████████| 1000/1000 [00:00<00:00, 39777.93it/s]
accuracy: 0.0003, loss: 5.0726 ||: 100%|██████████| 10/10 [00:00<00:00, 55.44it/s]
accuracy: 0.0000, loss: 4.8227 ||: 100%|██████████| 10/10 [00:00<00:00, 83.51it/s]
accuracy: 0.0812, loss: 4.6590 ||: 100%|██████████| 10/10 [00:00<00:00, 65.25it/s]
accuracy: 0.1328, loss: 4.4557 ||: 100%|██████████| 10/10 [00:00<00:00, 124.97it/s]
accuracy: 0.1333, loss: 4.3597 ||: 100%|██████████| 10/10 [00:00<00:00, 67.04it/s]
accuracy: 0.1328, loss: 4.2222 ||: 100%|██████████| 10/10 [00:00<00:00, 116.47it/s]
accuracy: 0.1333, loss: 4.1775 ||: 100%|██████████| 10/10 [00:00<00:00, 68.37it/s]
accuracy: 0.1328, loss: 4.0804 ||: 100%|██████████| 10/10 [00:00<00:00, 115.40it/s]
accuracy: 0.1333, loss: 4.0629 ||: 100%|██████████| 10/10 [00:00<00:00, 68.50it/s]
accuracy: 0.1328, loss: 3.9821 ||: 100%|██████████| 10/10 [00:00<00:00, 114.30it/s]
accuracy: 0.1333, loss: 3.9832 ||: 100%|██████████| 10/

{'best_epoch': 991,
 'best_validation_accuracy': 0.7383820998278829,
 'best_validation_loss': 1.29871267080307,
 'epoch': 999,
 'peak_cpu_memory_MB': 2289.376,
 'peak_gpu_0_memory_MB': 531,
 'training_accuracy': 0.9221146085552865,
 'training_cpu_memory_MB': 2289.376,
 'training_duration': '0:05:43.446376',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 531,
 'training_loss': 0.4671905905008316,
 'training_start_epoch': 0,
 'validation_accuracy': 0.7379039969401415,
 'validation_loss': 1.3000836849212647}

Now do the same training (500 sentence training and 500 sentence validation sets) with GloVE embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
glove_embedding_tiny_500 = Embedding.from_params(vocab=vocab_tiny_500, params=Params({'pretrained_file':'glove.6B.300d.txt', 'embedding_dim' : EMBEDDING_DIM}))
glove_model_tiny_500 = LstmTagger(BasicTextFieldEmbedder({"tokens": glove_embedding_tiny_500}), lstm, vocab_tiny_500)

glove_trainer_tiny_500 = initialize_trainer(glove_model_tiny_500,vocab_tiny_500,train_data_tiny_500,val_data_tiny_500,batch_size=5)
glove_trainer_tiny_500.train()

400000it [00:03, 108735.29it/s]
accuracy: 0.2540, loss: 3.9913 ||: 100%|██████████| 100/100 [00:00<00:00, 140.76it/s]
accuracy: 0.3344, loss: 3.3640 ||: 100%|██████████| 100/100 [00:00<00:00, 283.31it/s]
accuracy: 0.3728, loss: 3.1435 ||: 100%|██████████| 100/100 [00:00<00:00, 141.98it/s]
accuracy: 0.4304, loss: 2.9054 ||: 100%|██████████| 100/100 [00:00<00:00, 283.02it/s]
accuracy: 0.4410, loss: 2.7877 ||: 100%|██████████| 100/100 [00:00<00:00, 140.75it/s]
accuracy: 0.4545, loss: 2.6328 ||: 100%|██████████| 100/100 [00:00<00:00, 291.14it/s]
accuracy: 0.4655, loss: 2.5533 ||: 100%|██████████| 100/100 [00:00<00:00, 139.51it/s]
accuracy: 0.4801, loss: 2.4481 ||: 100%|██████████| 100/100 [00:00<00:00, 290.02it/s]
accuracy: 0.4924, loss: 2.3813 ||: 100%|██████████| 100/100 [00:00<00:00, 140.65it/s]
accuracy: 0.5004, loss: 2.3090 ||: 100%|██████████| 100/100 [00:00<00:00, 280.60it/s]
accuracy: 0.5271, loss: 2.2469 ||: 100%|██████████| 100/100 [00:00<00:00, 139.88it/s]
accuracy: 0.5286, loss

{'best_epoch': 101,
 'best_validation_accuracy': 0.7743354369860393,
 'best_validation_loss': 1.1115271443128585,
 'epoch': 110,
 'peak_cpu_memory_MB': 2302.364,
 'peak_gpu_0_memory_MB': 531,
 'training_accuracy': 0.9118240516545602,
 'training_cpu_memory_MB': 2302.364,
 'training_duration': '0:02:09.758626',
 'training_epochs': 110,
 'training_gpu_0_memory_MB': 531,
 'training_loss': 0.36729640915989875,
 'training_start_epoch': 0,
 'validation_accuracy': 0.7751004016064257,
 'validation_loss': 1.1148178628087044}

Use a bigger training set now with 5K sentence training and 5K sentence validation sets and random initial embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the basic model on 5000 sentences
train_data_tiny_5000 = reader.read("train_brown_5000.txt")
val_data_tiny_5000 = reader.read("validation_brown_5000.txt")

vocab_tiny_5000 = Vocabulary.from_instances(train_data_tiny_5000 + val_data_tiny_5000)

token_embedding_tiny_5000 = Embedding(vocab_tiny_5000.get_vocab_size(), EMBEDDING_DIM)

model_tiny_5000 = LstmTagger(BasicTextFieldEmbedder({"tokens": token_embedding_tiny_5000}), lstm, vocab_tiny_5000)

basic_trainer_tiny_5000 = initialize_trainer(model_tiny_5000,vocab_tiny_5000,train_data_tiny_5000,val_data_tiny_5000,batch_size=50)
basic_trainer_tiny_5000.train()

Streaming output truncated to the last 5000 lines.

  0%|          | 0/100 [00:00<?, ?it/s]
accuracy: 0.1714, loss: 3.4265 ||:   1%|          | 1/100 [00:00<00:14,  6.69it/s]
accuracy: 0.6468, loss: 1.9808 ||:  19%|█▉        | 19/100 [00:00<00:08,  9.41it/s]
accuracy: 0.6835, loss: 1.6633 ||:  35%|███▌      | 35/100 [00:00<00:04, 13.10it/s]
accuracy: 0.6940, loss: 1.5569 ||:  47%|████▋     | 47/100 [00:00<00:02, 17.84it/s]
accuracy: 0.6978, loss: 1.4902 ||:  59%|█████▉    | 59/100 [00:00<00:01, 23.91it/s]
accuracy: 0.7002, loss: 1.4477 ||:  70%|███████   | 70/100 [00:00<00:00, 31.24it/s]
accuracy: 0.7002, loss: 1.4186 ||:  81%|████████  | 81/100 [00:00<00:00, 39.49it/s]
accuracy: 0.6982, loss: 1.3906 ||: 100%|██████████| 100/100 [00:01<00:00, 98.92it/s]

  0%|          | 0/100 [00:00<?, ?it/s]
accuracy: 0.5779, loss: 2.2103 ||:   1%|          | 1/100 [00:00<00:14,  7.04it/s]
accuracy: 0.7391, loss: 1.3317 ||:   8%|▊         | 8/100 [00:00<00:09,  9.59it/s]
accuracy: 0.7416, loss: 1.352

{'best_epoch': 210,
 'best_validation_accuracy': 0.8449317738791423,
 'best_validation_loss': 0.7925410860776901,
 'epoch': 219,
 'peak_cpu_memory_MB': 2399.78,
 'peak_gpu_0_memory_MB': 665,
 'training_accuracy': 0.9222140075903199,
 'training_cpu_memory_MB': 2399.78,
 'training_duration': '0:10:55.503920',
 'training_epochs': 219,
 'training_gpu_0_memory_MB': 665,
 'training_loss': 0.39387426525354385,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8442007797270955,
 'validation_loss': 0.8100894749164581}

Now do the same training (5K sentence training and 5K sentence validation sets) with GloVE embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
glove_embedding_tiny_5000 = Embedding.from_params(vocab=vocab_tiny_5000, params=Params({'pretrained_file':'glove.6B.300d.txt', 'embedding_dim' : EMBEDDING_DIM}))
glove_model_tiny_5000 = LstmTagger(BasicTextFieldEmbedder({"tokens": glove_embedding_tiny_5000}), lstm, vocab_tiny_5000)

glove_trainer_tiny_5000 = initialize_trainer(glove_model_tiny_5000,vocab_tiny_5000,train_data_tiny_5000,val_data_tiny_5000,batch_size=5)
glove_trainer_tiny_5000.train()

Streaming output truncated to the last 5000 lines.
accuracy: 0.8148, loss: 1.0897 ||:  28%|██▊       | 277/1000 [00:01<00:05, 124.49it/s]
accuracy: 0.8206, loss: 1.0481 ||:  30%|███       | 301/1000 [00:01<00:04, 145.29it/s]
accuracy: 0.8237, loss: 1.0193 ||:  32%|███▎      | 325/1000 [00:01<00:04, 163.84it/s]
accuracy: 0.8272, loss: 0.9921 ||:  35%|███▍      | 349/1000 [00:01<00:03, 177.98it/s]
accuracy: 0.8295, loss: 0.9689 ||:  37%|███▋      | 372/1000 [00:01<00:03, 190.67it/s]
accuracy: 0.8300, loss: 0.9490 ||:  40%|███▉      | 396/1000 [00:01<00:02, 202.11it/s]
accuracy: 0.8326, loss: 0.9253 ||:  42%|████▏     | 421/1000 [00:01<00:02, 212.82it/s]
accuracy: 0.8351, loss: 0.9057 ||:  45%|████▍     | 446/1000 [00:02<00:02, 221.25it/s]
accuracy: 0.8374, loss: 0.8900 ||:  47%|████▋     | 470/1000 [00:02<00:02, 224.42it/s]
accuracy: 0.8385, loss: 0.8760 ||:  49%|████▉     | 494/1000 [00:02<00:02, 224.00it/s]
accuracy: 0.8391, loss: 0.8641 ||:  52%|█████▏    | 518/1000 [00:02<00:02, 218.

{'best_epoch': 50,
 'best_validation_accuracy': 0.8687621832358674,
 'best_validation_loss': 0.6885432055667042,
 'epoch': 59,
 'peak_cpu_memory_MB': 2440.48,
 'peak_gpu_0_memory_MB': 693,
 'training_accuracy': 0.9275370890630391,
 'training_cpu_memory_MB': 2440.48,
 'training_duration': '0:15:17.247773',
 'training_epochs': 59,
 'training_gpu_0_memory_MB': 693,
 'training_loss': 0.2681618054974824,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8706432748538012,
 'validation_loss': 0.6901265966072678}

For each trained model, record validation accuracy and training duration (they are returned along with other training stats after training a model) and accuracy on the training set. Fill in the numbers in the table below:

| model | validation accuracy | training accuracy | training duration|
|-------|---------------------|---------------|-------------------------------------------
| basic model on 50 sentences|0.38155136268343814|0.3903192584963955|0:02:26.720229|
| glove model on 50 sentences|0.4790356394129979|0.6570545829042225|0:02:26.034040|
| basic model on 500 sentences|0.7379039969401415|0.9221146085552865|0:05:43.446376|
| glove model on 500 sentences|0.7751004016064257|0.9118240516545602|0:02:09.758626|
| basic model on 5000 sentences|0.8442007797270955|0.9222140075903199|0:10:55.503920|
| glove model on 5000 sentences|0.8706432748538012|0.9275370890630391|0:15:17.247773|

**Question.** What do you conclude from these comparisons? when can it be especially beneficial to initialize a model with pretrained embeddings?

From the table it could be observed that in all cases the Glove model has a higher accuracy in both training and validation. Moreover, the basic model does benefit most of a larger dataset, from 50 to 500 the accuracy is doubled in both training and validation. The models trained on 5000 sentences has a higher accruacy but the traning duration is also increased drastically.

When initiating a pretrained embedding it would be highly beneficial to have a high similarity between the data it is trained on and the dataset. Moreover, a small dataset is sufficient to use the embedding.

## Comment 
In this lab we used pretrained GloVe embeddings in a model for part of speech tagging. GloVe in its turn is also a neural word embedding model, but it had been trained on a completely different objective. GloVe vectors had been optimised on word cooccurrence matrix decomposition, i.e. on the task of predicting which words tend to occur with which other words. Part of speech certainly plays a role in determining statistical cooccurrence of words, but this role is indirect, and explicit part of speech information has not been used in training GloVe.

This makes our application an example of **transfer learning**, whereby a learned model trained on one objective (e.g. word cooccurrence) can benefit a different application (e.g. POS tagging), because some information is shared between them. 

## Part 2 - ELMo vectors (55 points)

> Indented block



In the second part of this lab we will reproduce the word sense disambiguation strategy that the authors of the ELMo vectors explored. The strategy consists in the following:

- create ELMo embeddings for all tokens in a sense-annotated corpus
- calculate mean sense vectors for each word sense in the training partition of the corpus
- for each sense-annotated token in the test partition of the corpus, assign it to the sense of the word to which its ELMo vector is the closest according to the cosine distance metric
- as a backup strategy, use the 1st sense of the word by default.

As a sense annotated corpus, we can use SemCor, conveniently available within NLTK. <code>semcor.sents()</code> iterates over all sentences represented as lists of tokens, while <code>semcor.tagged_sents()</code> iterates over the same sentences with additional annotation including WordNet lemma identifiers (lemmas in WordNet stand for a word taken in a specific sense).

In [15]:
import nltk
nltk.download('wordnet')
nltk.download('semcor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package semcor to /root/nltk_data...


True

In [16]:
from nltk.corpus import wordnet as wn  
from nltk.corpus import semcor
semcor.sents()
semcor.tagged_sents(tag="sem")

[[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']], [['The'], Tree(Lemma('jury.n.01.jury'), ['jury']), Tree(Lemma('far.r.02.far'), ['further']), Tree(Lemma('state.v.01.say'), ['said']), ['in'], Tree(Lemma('term.n.02.term'), ['term']), Tree(Lemma('end.n.02.end'), ['end']), Tree(Lemma('presentment.n.01.presentment'), ['presentments']), ['

## Exercise 1. Extract relevant data from SemCor (5 points)

Let's prepare SemCor data for the disambiguation task. Since this is just an educational exercise and we don't aim at replicating the full results, we can use only a subset of SemCor. Take the first 10K sentences of SemCor and split them randomly into 90% training and 10% testing partitions:

In [22]:
sentences = list(semcor.tagged_sents(tag="sem"))

In [23]:
semcor_train = sentences[:9000]
semcor_test = sentences[9000:10000]

Create a function that takes as input a sentence from SemCor and extracts a list which contains, for each token of the sentence, either the corresponding WordNet Lemma (e.g. <code>Lemma('friday.n.01.Friday')</code>) or <code>None</code>. <code>None</code> corresponds to tokens that are either 1) not annotated for word senses (e.g. articles); 2) are marked up as (part of) a named entity (e.g. "City of Atlanta" or placename "Fulton" annotated as  <code>Tree(Lemma('location.n.01.location'), [Tree('NE', ['Fulton'])])</code>). For example, <code>get_lemmas</code> for the first sentence of Semcor should return <code>[None,
 None,
 None,
 None,
 None,
 Lemma('state.v.01.say'),
 Lemma('friday.n.01.Friday'),
 None,
 Lemma('probe.n.01.investigation'),
 None,
 Lemma('atlanta.n.01.Atlanta'),
 None,
 Lemma('late.s.03.recent'),
 Lemma('primary.n.01.primary_election'),
 Lemma('primary.n.01.primary_election'),
 Lemma('produce.v.04.produce'),
 None,
 None,
 Lemma('evidence.n.01.evidence'),
 None,
 None,
 None,
 Lemma('abnormality.n.04.irregularity'),
 Lemma('happen.v.01.take_place'),
 Lemma('happen.v.01.take_place'),
 None]</code>

In [24]:
def get_lemmas(semcor_sentence):
  return [token.label() if isinstance(token, nltk.tree.Tree) else None for token in semcor_sentence]

You are now able to extract word senses (instantiated by WordNet lemmas) from the corpus. The next step is to associate senses with ELMo vectors. Create a dictionary of contextualized token embeddings from the training corpus grouped by the WordNet sense:

In [20]:
from collections import defaultdict

Train_embeddings=defaultdict(list)

Now let's create contextualized ELMo word embeddings for the tokens in this corpus. We can load the pretrained ELMo model and define a function <code>sentences_to_elmo()</code> that receives a list of tokenized sentences as input and produces their ELMo vectors.

In [6]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0)

def sentences_to_elmo(sentences):
    character_ids = batch_to_ids(sentences)
    embeddings = elmo(character_ids)
    return embeddings

100%|██████████| 374434792/374434792 [00:22<00:00, 16398327.38B/s]


Now you can process the corpus sentences and produce their ELMo vectors. It is recommended to pass the input to ELMo encoder in batches. A suggested batch size is 50 sentences. For example, the code below processes the first 50 sentences from the corpus:

In [17]:
sentences=semcor.sents()[:50]
print(sentences)
embeddings=sentences_to_elmo(sentences)

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term', 'end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ['The', 'September', 'October', 'term', 'jury', 'had', 'been', 'charged', 'by', 'Fulton', 'Superior', 'Court', 'Judge', 'Durwood', 'Pye', 'to', 'investigate', 'reports', 'of', 'possible', '``', 'irregularities', "''", 'in', 'the', 'hard-fought', 'primary', 'which', 'was', 'won', 'by', 'Mayor-nominate', 'Ivan', 'Allen', 'Jr.', '.'], ['``', 'Only', 'a', 'relative', 'handful', 'of', 's

The <code>embeddings</code> that we obtained is a dictionary that contains a list of ELMo embeddings and a list of masks. The mask tells us which embeddings correspond to tokens in the original input sentences and which correspond to the padding (introduced to give all sentences in the batch the same length).
In principle all embeddings are stored in PyTorch tensors so that they can be used in bigger neural models, but we are not going to do it now.

In [51]:
embeddings['elmo_representations'][0]

tensor([[[-6.4618e-03,  6.0215e-03, -3.5598e-01,  ..., -1.1715e-02,
           7.0427e-02, -4.1873e-01],
         [-3.7781e-01,  2.8141e-01, -2.5836e-01,  ..., -4.8547e-01,
           2.5508e-01,  3.6381e-02],
         [ 9.1191e-01,  1.1779e+00, -8.4833e-01,  ...,  9.8472e-01,
           3.3675e-01,  1.6172e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-6.4618e-03,  6.0215e-03, -3.5598e-01,  ..., -4.4876e-02,
           1.1313e-01, -9.9628e-02],
         [ 1.3721e-01, -2.0003e-01, -1.3074e-01,  ...,  5.9482e-01,
           9.3387e-01, -2.6757e-01],
         [ 1.7280e-01,  1.0801e+00, -5.4539e-01,  ...,  3.1966e-01,
          -5.6408e-01,  3.2461e-01],
         ...,
         [ 0.0000e+00,  0

We can check the size of the embeddings we got. It has three dimensions: 1) the number of sentences 2) the number of tokens (corresponds to the tokens in the longest original sentence of the batch; shorter ones were padded) and 3) the dimensionality of the Elmo vector (1024).

In [52]:
embeddings['elmo_representations'][0].detach().size()

torch.Size([50, 59, 1024])

Another thing contained in the <code>embeddings</code> is the mask, a tensor encoding which token vectors correspond to original tokens and which are paddings. It has two dimensions, one corresponding to the sentences in the batch (50) and one corresponding to the token positions:

In [53]:
print(embeddings['mask'].size())
embeddings['mask']

torch.Size([50, 59])


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

## Exercise 2. Extract ELMo encoding of sentences using a mask (5 points)  

Now define a function <code>get_masked_vectors(embeddings)</code> that takes embeddings as input and returns a list of ELMo sentence encodings to which the mask has been applied.  The output should be a list of Torch tensors, where the padding vectors have been removed so each sentence is represented by an nx1024 tensor where n is sentence length.

In [18]:
def get_masked_vectors(embeddings):
    sentences = list()
    embedding = embeddings['elmo_representations'][0].detach().numpy()
    for i in range(len(embeddings['mask'])):
      sentences.append(embedding[i,:max(np.where(embeddings['mask'][i] == 1)[0])+1])

    return sentences

## Exercise 3. Collect ELMo vectors from the training corpus (20 points)

Process the corpus updating your train word sense vectors in the dictionary. Iterate over the all the train sentences in the corpus, and retrieve for each lemma-annotated token (where lemma is not <code>None</code>) the corresponding ELMo vector. Store the ELMo sense embeddings that correspond to each lemma in the dictionary <code>Train_embeddings</code>. This step of processing the training corpus with ELMo is the most time consuming part of this assignment. However, it should not take forever. If this computation takes more than an hour, you may want to optimize your code or make sure you are using GPU acceleration. For the purposes of developing and debugging your solution, you may start by use a sample of 100 sentences, but then switch to the full 9K sentence training set. 

In [ ]:
Train_embeddings = defaultdict(list)
for sentence in semcor_train:
  encoded_sentence = list()
  new_sentence = list()
  lemmas = [i for i in get_lemmas(sentence) if i is not None and type(i) is not str]
  for lemma in lemmas:
    if lemma is not None and type(lemma) is not str:
      string = str(lemma)
      string = string.split(".")[0].replace("Lemma('","")
      new_sentence.append(string)

  encoded_sentence = get_masked_vectors(sentences_to_elmo(new_sentence)) 

  for (lemma, encoding) in zip(lemmas, encoded_sentence):
    Train_embeddings[lemma].append(encoding)

How many senses does your Train_embeddings contain? **Do not clear the output of this cell in your submission**

In [26]:
print(len(Train_embeddings))

604


## Exercise 4. Vector averaging (5 points)

Your <code>Train_embeddings</code> now is a list of all vectors for a given word sense in the training corpus. For our purposes, we do not need the full list but the mean vector for each sense. For each sense in <code>Train_embeddings</code>, substitute the list by the average ELMo vector on the list. One efficient way to do this is to convert the list to a tensor via <code>stack</code> function and use Torch's <code>mean</code> function. Below is an example of how an average of two (random) vectors stored in a tensor can be computed in PyTorch: 

In [219]:
 randtensor= torch.randn(2, 4)
 print("Tensor storing two 4-dimensional vectors:\n",randtensor)
 print("Average vector: \n",randtensor.mean(dim=0))

Tensor storing two 4-dimensional vectors:
 tensor([[-1.0635,  1.6892,  1.1862, -1.5068],
        [-1.0691, -0.7000, -0.2373, -0.4902]])
Average vector: 
 tensor([-1.0663,  0.4946,  0.4745, -0.9985])


Now you are ready to update your <code>Train_embeddings</code> so that it maps lemmas not to lists but to averaged vectors.

In [61]:
for lemma, vectors in Train_embeddings.items():
  vecs = list()
  for vector in vectors:
    vecs.append(torch.mean(torch.tensor(vector).float(),dim=0))

  Train_embeddings[lemma] =  vecs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


## Exercise 5. Testing the sense vectors (20 points)

Test your sense embeddings on your test data, which is a subset of the SemCor corpus. Use the strategy outlined above, with 1st WordNet sense as a fallback: 

- rely on mean sense vectors for each word sense in the training partition of the corpus, as stored in <code>Train_embeddings</code>
- for each sense-annotated token <i>t</i> (e.g. the verb "run") in the test partition of the corpus, assign it to the sense of the word "Lemma('X.v.n.run')" to which ithe ELMo vector <i>t</i> is the closest according to the cosine distance metric
- as a backup strategy, use the 1st sense of the word (e.g. <code>Lemma('run.n.01.run')</code>) from WordNet. You can look it up using a built-in function from NLTK (e.g. <code>wn.lemmas('run')</code>).

Calculate WSD accuracy in percentage points on your test data. Report three numbers
- overall accuracy (proportion of times the ELMo method+WordNet backup results in the correct sense annotation)
- WordNet baseline accuracy: what if you always select the first WordNet sense, ignoring the ELMo embedding?
- accuracy of the ELMo method just for the instances in which ELMo strategy is applicable
- accuracy of the WordNet baseline just for the instances in which ELMo strategy is applicable

 **Do not delete the output of this cell in the submitted version**

In [ ]:
from torch.nn.functional import cosine_similarity

# Your code here
print("Overall accuracy:", accuracy) 
print("WordNet baseline:", baseline_accuracy)
print("Accuracy in cases where ELMo method is used", elmo_accuracy)
print("Accuracy of the baseline in cases where ELMo method is applicable", baseline_on_elmo_data_accuracy)


If you reached this point, you were able to evaluate ELMo as a model of contextual semantic similarity of word usages. The idea behind the vector averaging is that a word when used in the same sense should have similar vector representations, while usages in distinct senses should have different vector representations.

Analyze the numbers above. What do they tell you?

**WRITE YOUR ANALYSIS HERE**


## The end
Congratulations! this is the end of Lab 4.